<a href="https://colab.research.google.com/github/YamRub/NLP-with-Sequence-model/blob/main/TopicAssignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [247]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
import string

In [220]:
df = pd.read_excel("/content/sample_data/netflix_reviews_NLP Sample Data.xlsx")

In [298]:
df1=df.copy(deep=True)

In [243]:
df1.head()

,reviewId,userName,content,score,thumbsUpCount,reviewCreatedVersion,at,appVersion
0,c01a0242-e5e7-4e8a-a948-3f0b3ba89f20,Chris Paredes,Where it's me on the plane. It's good. You can...,5,0,8.113.3 build 31 50678,2024-05-21 22:41:10,8.113.3 build 31 50678
1,3c1c9fab-65e9-4f89-a1b6-9c45f27addb1,ijustwannabefab,I have an account for 5 users that I share wit...,1,1,8.114.0 build 19 50680,2024-05-21 22:25:36,8.114.0 build 19 50680
2,27fdf06b-24e8-4a72-9092-c88518e2ceb2,Darah Lazo,Can never go a day without Netflix ðŸ˜Ž,5,0,8.114.0 build 19 50680,2024-05-21 22:25:29,8.114.0 build 19 50680
3,2e8efd65-9b51-4ce4-ace0-411215764a93,jeremy porter,And another streaming service that I'm not gon...,1,1,8.113.3 build 31 50678,2024-05-21 22:25:17,8.113.3 build 31 50678
4,5918b74d-38d0-4e9a-ac5e-da2738351352,Jacob Overcash,Cannot cast if you have ad level plan. Worthless,1,0,8.114.0 build 19 50680,2024-05-21 22:10:44,8.114.0 build 19 50680


# New Section

In [223]:
df1.columns

Index(['reviewId', 'userName', 'content', 'score', 'thumbsUpCount',
       'reviewCreatedVersion', 'at', 'appVersion'],
      dtype='object')

In [38]:
df1['content'][0]

"Where it's me on the plane. It's good. You can watch movit's really great. I love it and I'm going to go on a plane in 2 and a 1/2 minutes."

In [39]:
df1.dtypes

reviewId                        object
userName                        object
content                         object
score                            int64
thumbsUpCount                    int64
reviewCreatedVersion            object
at                      datetime64[ns]
appVersion                      object
dtype: object

In [299]:
df1['content'] = df1['content'].astype(str)

In [225]:
# # Preprocess the reviews
# def preprocess_text(text):
#     #text = text.lower()  # lowercase
#     text=  re.sub(r'[^a-zA-Z]','',text)
#     text = re.sub(r'\d+', '', text)  # remove digits
#     text = re.sub(r'[^\w\s]', '', text)  # remove punctuation
#     text = re.sub(r'\s+', ' ', text).strip()  # remove extra whitespace
#     return text

In [300]:
def preprocess_text(text):
    # Initialize stemmer and stopwords
    stemmer = PorterStemmer()
    stop_words = set(stopwords.words('english'))

    # Remove punctuation
    text = re.sub(f"[{re.escape(string.punctuation)}]", "", text)
    # Convert to lower case
    text = text.lower()
    # Remove stop words and perform stemming
    text = " ".join([stemmer.stem(word) for word in text.split() if word not in stop_words])
    return text

In [301]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [302]:
df1['content']=df1['content'].apply(preprocess_text)

In [140]:
df1['score'].describe()

count    109729.000000
mean          2.807107
std           1.695414
min           1.000000
25%           1.000000
50%           3.000000
75%           5.000000
max           5.000000
Name: score, dtype: float64

In [110]:
#df1['sentiment'] = df1['score'].apply(lambda x: 1 if x >= 3 else 0) # creating the 'sentiment'

In [93]:
import plotly.express as px
def plot_histogram(dataframe, col, nbins=None, color=None, bargap=None, text=False, title=None, xlabel=None):
    """
    This function takes as input a dataframe and plots the histogram of the specified column.

    Args:
        dataframe = A pandas dataframe,
        col = The column of the dataframe for which you want to plot the histogram,
        nbins = Number of bins in the histogram,
        color = The column of the dataframe based on whose values you want the bars of the histogram to be colored,
        bargap = Gap among the histogram bars,
        text = Whether to display the values on the histogram bars,
        title = The plot title,
        xlabel = The title of the x-axis
    """

    fig = px.histogram(dataframe,
                       x=dataframe[col].map(len) if dataframe[col].dtype != int else col,
                       nbins=nbins if nbins else None,
                       text_auto=text if text else False,
                       color=color if color else None,
                       title=title if title else None)
    fig.update_layout(bargap=bargap if bargap else None)
    fig.update_xaxes(title_text=xlabel if xlabel else None)
    fig.show()

In [98]:
plot_histogram(df1, 'content', color='sentiment', title='Histogram of review lengths',
               xlabel='Length of reviews in terms of characters')

In [100]:
plot_histogram(df1, 'score', color='score', text=True, bargap=0.25, title='Count of scores', xlabel='Scores')

In [101]:
plot_histogram(df1, 'sentiment', color='sentiment', bargap=0.25, title='Count of sentiments', xlabel='Sentiment', text=True)

In [303]:
df1.columns

Index(['reviewId', 'userName', 'content', 'score', 'thumbsUpCount',
       'reviewCreatedVersion', 'at', 'appVersion'],
      dtype='object')

In [304]:
df1 = df1.drop(df1.columns[[0, 1,3, 4, 5, 6, 7]], axis=1)

In [274]:
#df1 = df1.drop(df1.columns[[1,2]], axis=1)

In [305]:
df1.head()

,content
0,plane good watch movit realli great love im go...
1,account 5 user share dad alway use playstat da...
2,never go day without netflix ðÿ˜ž
3,anoth stream servic im gonna pay anymor want p...
4,cannot cast ad level plan worthless


In [261]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109729 entries, 0 to 109728
Data columns (total 1 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   content  109729 non-null  object
dtypes: object(1)
memory usage: 857.4+ KB


In [306]:
# Manually labeling data for demonstration
# Create a small sample of labeled data for testing purposes
sample_data = {
    'content': [
        'The price point is too high for the service offered.',
        'I love the multi-user feature, it allows me to share my account with my family.',
        'The streaming quality is excellent, no buffering at all.',
        'The selection of shows is amazing, very diverse and entertaining.',
        'Great collection of movies, very satisfied with the offerings.',
        'The content is well organized, easy to find what I am looking for.'
    ],
    'label': [
        'cost of service and price point',
        'multiuser and account sharing',
        'streaming quality',
        'shows offerings',
        'movie offerings',
        'content organization'
    ]
}

In [307]:
# Append the sample data to the dataframe
sample_df = pd.DataFrame(sample_data)


In [309]:
sample_df.shape

(6, 2)

In [310]:
df1 = pd.concat([df1, sample_df], ignore_index=True)

In [311]:
df1

,content,label
0,plane good watch movit realli great love im go...,NaN
1,account 5 user share dad alway use playstat da...,NaN
2,never go day without netflix ðÿ˜ž,NaN
3,anoth stream servic im gonna pay anymor want p...,NaN
4,cannot cast ad level plan worthless,NaN
...,...,...
109730,"I love the multi-user feature, it allows me to...",multiuser and account sharing
109731,"The streaming quality is excellent, no bufferi...",streaming quality
109732,"The selection of shows is amazing, very divers...",shows offerings
109733,"Great collection of movies, very satisfied wit...",movie offerings


In [312]:
topics = ["cost of service and price point", "multiuser and account sharing",
          "streaming quality", "shows offerings", "movie offerings", "content organization"]

In [314]:
topic_to_id = {topic: idx for idx, topic in enumerate(topics)}
id_to_topic = {idx: topic for topic, idx in topic_to_id.items()}

# Only map labels that are in our sample data
df1['label_id'] = df1['label'].map(topic_to_id)

# Ensure only rows with non-NaN labels are used for training
df_labeled = df1.dropna(subset=['label_id'])

In [315]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109735 entries, 0 to 109734
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   content   109735 non-null  object 
 1   label     6 non-null       object 
 2   label_id  6 non-null       float64
dtypes: float64(1), object(2)
memory usage: 2.5+ MB


In [320]:
# Function to classify reviews
def classify_reviews(df_labeled, df, topics):
    # Vectorize the reviews using TF-IDF
    vectorizer = TfidfVectorizer(max_features=5000)
    X = vectorizer.fit_transform(df_labeled['content'])
    y = df_labeled['label_id'].astype(int)

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train a classifier (using Logistic Regression for simplicity)
    classifier = LogisticRegression(max_iter=1000)
    classifier.fit(X_train, y_train)

    # Evaluate the model
    y_pred = classifier.predict(X_test)
    print("Classification Report:\n", classification_report(y_test, y_pred, labels=np.unique(y_train), target_names=[id_to_topic[id] for id in np.unique(y_train)]))

    # Predict on the entire dataset (including unlabeled)
    X_all = vectorizer.transform(df['content'])
    df['predicted_label_id'] = classifier.predict(X_all)
    df['predicted_label'] = df['predicted_label_id'].map(id_to_topic)

    # Present the results
    result = df[['content', 'label', 'predicted_label']]
    return result


In [321]:
# Classify reviews
classification_result = classify_reviews(df_labeled,df1, topics)

# Display the classification result
print(classification_result.head())

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this

Classification Report:
                       precision    recall  f1-score   support

   streaming quality       0.00      0.00      0.00       0.0
     shows offerings       0.00      0.00      0.00       0.0
     movie offerings       0.00      0.00      0.00       0.0
content organization       0.00      0.00      0.00       0.0

           micro avg       0.00      0.00      0.00       0.0
           macro avg       0.00      0.00      0.00       0.0
        weighted avg       0.00      0.00      0.00       0.0

                                             content label  \
0  plane good watch movit realli great love im go...   NaN   
1  account 5 user share dad alway use playstat da...   NaN   
2                  never go day without netflix ðÿ˜ž   NaN   
3  anoth stream servic im gonna pay anymor want p...   NaN   
4                cannot cast ad level plan worthless   NaN   

        predicted_label  
0       movie offerings  
1  content organization  
2  content organization  


In [324]:
classification_result.to_csv('/content/sample_data/preprocessed_reviews1.csv', index=False)

In [335]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df_labeled['encoded_label'] = label_encoder.fit_transform(df_labeled['label_id'].astype(int))

<ipython-input-335-6b9a9d0b2e56>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [338]:
def classify_reviews_rf(df_labeled, df, topics):
    vectorizer = TfidfVectorizer(max_features=5000)
    X = vectorizer.fit_transform(df_labeled['content'])
    y = df_labeled['encoded_label']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
    rf_classifier.fit(X_train, y_train)

    y_pred = rf_classifier.predict(X_test)
    print("Random Forest Classification Report:\n", classification_report(y_test, y_pred, target_names=label_encoder.classes_))

    X_all = vectorizer.transform(df['content'])
    df['rf_predicted_label_id'] = label_encoder.inverse_transform(rf_classifier.predict(X_all))
    df['rf_predicted_label'] = df['rf_predicted_label_id'].map(id_to_topic)

    return df[['content', 'label', 'rf_predicted_label']]

In [340]:
# Classify reviews using Random Forest
rf_classification_result = classify_reviews_rf(df_labeled, df, topics)
print(rf_classification_result.head())

ValueError: Number of classes, 4, does not match size of target_names, 6. Try specifying the labels parameter

In [328]:
import xgboost as xgb

In [333]:
def classify_reviews_xgb(df_labeled, df, topics):
    vectorizer = TfidfVectorizer(max_features=5000)
    X = vectorizer.fit_transform(df_labeled['content'])
    y = df_labeled['encoded_label']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    xgb_classifier = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
    xgb_classifier.fit(X_train, y_train)

    y_pred = xgb_classifier.predict(X_test)
    print("XGBoost Classification Report:\n", classification_report(y_test, y_pred, target_names=label_encoder.classes_))

    X_all = vectorizer.transform(df['content'])
    df['xgb_predicted_label_id'] = label_encoder.inverse_transform(xgb_classifier.predict(X_all))
    df['xgb_predicted_label'] = df['xgb_predicted_label_id'].map(id_to_topic)

    return df[['content', 'label', 'xgb_predicted_label']]


In [334]:
# Classify reviews using XGBoost
xgb_classification_result = classify_reviews_xgb(df_labeled, df, topics)
print(xgb_classification_result.head())

ValueError: Invalid classes inferred from unique values of `y`.  Expected: [0 1 2 3], got [2 3 4 5]